In [1]:
import pandas as pd
import numpy as np
from torch import nn
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


In [2]:
champions_df = pd.read_csv('processed_data/champions_df.csv')
items_df = pd.read_csv('processed_data/items_df.csv')
other_features_df = pd.read_csv('processed_data/other_features_df.csv')
item_encoder = pd.read_pickle('processed_data/item_encoder.pkl')
champion_encoder = pd.read_pickle('processed_data/champion_encoder.pkl')

In [3]:
# define item file and get items name from the json
import json

with open('item.json', 'r') as file:
    items = json.load(file)


items['data']['1001']


{'name': 'Boots',
 'description': '<mainText><stats><attention>25</attention> Move Speed</stats><br><br></mainText>',
 'colloq': ';',
 'plaintext': 'Slightly increases Move Speed',
 'into': ['3005',
  '3047',
  '3006',
  '3009',
  '3010',
  '3020',
  '3111',
  '3117',
  '3158'],
 'image': {'full': '1001.png',
  'sprite': 'item0.png',
  'group': 'item',
  'x': 0,
  'y': 0,
  'w': 48,
  'h': 48},
 'gold': {'base': 300, 'purchasable': True, 'total': 300, 'sell': 210},
 'tags': ['Boots'],
 'maps': {'11': True,
  '12': True,
  '21': True,
  '22': False,
  '30': False,
  '33': False},
 'stats': {'FlatMovementSpeedMod': 25}}

In [20]:
# since we are working with multi label classification I will try to work with NN

class LoLItemizationModel(nn.Module):
    def __init__(self, num_champions, num_other_features, embedding_dim=32, hidden_dim=64, output_dim=100):
        super(LoLItemizationModel, self).__init__()
        
        # Champion embedding layer
        self.champion_embedding = nn.Embedding(num_champions, embedding_dim)
        
        # Layers for processing champion embeddings
        self.champion_layers = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Layers for processing other features and combining with champion features
        self.combined_layers = nn.Sequential(
            nn.Linear(hidden_dim + num_other_features, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()
        )
        
    def forward(self, champion_ids, other_features):
        # Process champion embeddings
        champion_embedded = self.champion_embedding(champion_ids)
        champion_features = self.champion_layers(champion_embedded)
        
        # Combine champion features with other features
        combined = torch.cat([champion_features, other_features], dim=1)
        
        # Process combined features
        output = self.combined_layers(combined)
        
        return output

In [4]:
from sklearn.model_selection import train_test_split


X_champion = torch.LongTensor(champions_df['champion_id'].values)
X_other = torch.FloatTensor(other_features_df.values)
y = torch.FloatTensor(items_df.values)

X_champion_temp, X_champion_test, X_other_temp, X_other_test, y_temp, y_test = train_test_split(
    X_champion, X_other, y, test_size=0.2, random_state=42
)

# second split: split the remaining data into train and validation sets
X_champion_train, X_champion_val, X_other_train, X_other_val, y_train, y_val = train_test_split(
    X_champion_temp, X_other_temp, y_temp, test_size=0.25, random_state=42  # 0.25 * 0.8 = 0.2
)

In [ ]:


train_data = TensorDataset(X_champion_train, X_other_train, y_train)
val_data = TensorDataset(X_champion_val, X_other_val, y_val)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

# Instantiate the model
num_champions = len(champion_encoder.classes_)
num_other_features = other_features_df.shape[1]
output_dim = items_df.shape[1]

model = LoLItemizationModel(num_champions, num_other_features, output_dim=output_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for champion_ids, other_features, labels in train_loader:
        champion_ids, other_features, labels = champion_ids.to(device), other_features.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(champion_ids, other_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * champion_ids.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for champion_ids, other_features, labels in val_loader:
            champion_ids, other_features, labels = champion_ids.to(device), other_features.to(device), labels.to(device)
            outputs = model(champion_ids, other_features)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * champion_ids.size(0)
    
    val_loss /= len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'lol_itemization_model.pth')

print("Training complete. Model saved as 'lol_itemization_model.pth'")

In [21]:
# let's define a function that takes in item number and outputs the item name

def get_item_name(item_number, item_encoder, items):
    # Create a binary vector for the single item
    binary_vector = np.zeros((1, len(item_encoder.classes_)))
    binary_vector[0, item_number] = 1
    
    # Use inverse_transform on the binary vector
    item_id = item_encoder.inverse_transform(binary_vector)[0][0]
    item_name = items['data'][str(item_id)]['name']
    return item_name

# write a function that tests model on a single example from test set

def test_model(model, test_loader):

    model.eval()
    with torch.no_grad():
        champion_ids, other_features, labels = next(iter(test_loader))
        champion_ids, other_features, labels = champion_ids.to(device), other_features.to(device), labels.to(device)
        outputs = model(champion_ids, other_features)
        predicted_items = outputs.cpu().numpy()
    
    # Print the predicted items
    print("Predicted items:")
   
    for i, item in enumerate(predicted_items[0]):
        if item >= 0.2:
            print(get_item_name(i,item_encoder, items))

def load_model(file_path, num_champions, num_other_features, output_dim):
    model = LoLItemizationModel(num_champions, num_other_features, output_dim=output_dim)
    
    # Load the state dict
    state_dict = torch.load(file_path)
    
    model.load_state_dict(state_dict)
    
    model.eval()
    
    return model


# Test the model

def decode_input(X_champion, X_other, champion_encoder, other_features_df):
    # Decode champion
    champion_id = X_champion.numpy()[0]
    champion_name = champion_encoder.inverse_transform([champion_id])[0]
    
    # Decode other features
    other_features = pd.DataFrame(X_other.numpy(), columns=other_features_df.columns)
    
    return champion_name, other_features

num_champions = len(champion_encoder.classes_)
num_other_features = other_features_df.shape[1]
output_dim = items_df.shape[1]

champion_name, other_features= decode_input(X_champion_val[:1], X_other_val[:1],champion_encoder, other_features_df)

print("Champion:", champion_name)
print("\nOther Features:")
print(other_features)

#read the model from pth
model = load_model('lol_itemization_model.pth', num_champions, num_other_features, output_dim)
print(X_champion_val[:1], X_other_val[:1], y_val[:1])


Champion: Jayce

Other Features:
   Enemy Top_AD_Melee_Assassin  Enemy Top_AD_Melee_Bruiser  \
0                          0.0                         0.0   

   Enemy Top_AD_Melee_Tank  Enemy Top_AD_Range_Shooter  \
0                      0.0                         0.0   

   Enemy Top_AP_Melee_Assassin  Enemy Top_AP_Melee_Bruiser  \
0                          0.0                         0.0   

   Enemy Top_AP_Melee_Tank  Enemy Top_AP_Range_Assassin  \
0                      1.0                          0.0   

   Enemy Top_AP_Range_Mage  Enemy Top_nan  ...  Secondary Rune_8446  \
0                      0.0            0.0  ...                  0.0   

   Secondary Rune_8451  Secondary Rune_8453  Secondary Rune_8463  \
0                  0.0                  0.0                  0.0   

   Secondary Rune_8473  Secondary Rune_9101  Secondary Rune_9103  \
0                  0.0                  0.0                  0.0   

   Secondary Rune_9104  Secondary Rune_9105  Secondary Rune_9111

C:\Users\kaczm\AppData\Local\Temp\ipykernel_3228\2611523905.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(file_path)


In [23]:
val_data = TensorDataset(X_champion_val[:1], X_other_val[:1], y_val[:1])
test_loader = DataLoader(val_data, batch_size=1)
test_model(model, test_loader)

for i in range(9, 13):  
    print(f"Example {i}:")
    champion_name, other_features= decode_input(X_champion_val[i-1:i], X_other_val[i-1:i],champion_encoder, other_features_df)

    print("Champion:", champion_name)
    val_data = TensorDataset(X_champion_val[i-1:i], X_other_val[i-1:i], y_val[i-1:i])
    test_loader = DataLoader(val_data, batch_size=1)
    test_model(model, test_loader)

Predicted items:
Long Sword
Doran's Blade
Muramana
Youmuu's Ghostblade
Ionian Boots of Lucidity
Farsight Alteration
Oracle Lens
Eclipse
Serylda's Grudge
Example 9:
Champion: Caitlyn
Predicted items:
Cloak of Agility
Doran's Blade
Berserker's Greaves
Infinity Edge
Rapid Firecannon
Farsight Alteration
Galeforce
Example 10:
Champion: Renekton
Predicted items:
Poro-Snax
Plated Steelcaps
Spirit Visage
Black Cleaver
Mercury's Treads
Death's Dance
Goredrinker
Example 11:
Champion: Yasuo
Predicted items:
Cloak of Agility
B. F. Sword
Doran's Blade
Berserker's Greaves
Infinity Edge
Bloodthirster
Zeal
Stealth Ward
Farsight Alteration
Oracle Lens
Immortal Shieldbow
Example 12:
Champion: Jhin
Predicted items:
Cloak of Agility
Boots of Swiftness
Infinity Edge
Rapid Firecannon
Farsight Alteration
Galeforce


In [5]:
class LoLItemizationModel_v2(nn.Module):
    def __init__(self, num_champions, num_other_features, output_dim, embedding_dim=32, hidden_dim=64, dropout_rate=0.5):
        super(LoLItemizationModel_v2, self).__init__()  
        

        self.champion_embedding = nn.Embedding(num_champions, embedding_dim)
        
        self.champion_layers = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        
        self.combined_layers = nn.Sequential(
            nn.Linear(hidden_dim + num_other_features, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()
        )
        
    def forward(self, champion_ids, other_features):
        champion_embedded = self.champion_embedding(champion_ids)
        champion_features = self.champion_layers(champion_embedded)

        combined = torch.cat([champion_features, other_features], dim=1)
        
        output = self.combined_layers(combined)
        
        return output

In [ ]:
X_champion = torch.LongTensor(champions_df['champion_id'].values)
X_other = torch.FloatTensor(other_features_df.values)
y = torch.FloatTensor(items_df.values)

learning_rate = 0.0005

# Split the data
X_champion_train, X_champion_val, X_other_train, X_other_val, y_train, y_val = train_test_split(
    X_champion, X_other, y, test_size=0.2, random_state=42
)

# Create DataLoader
train_data = TensorDataset(X_champion_train, X_other_train, y_train)
val_data = TensorDataset(X_champion_val, X_other_val, y_val)

batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)

num_champions = len(champion_encoder.classes_)
num_other_features = other_features_df.shape[1]
output_dim = items_df.shape[1]

model_v2 = LoLItemizationModel_v2(num_champions, num_other_features, output_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model_v2.parameters(), lr=learning_rate)

# Training loop
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_v2.to(device)

for epoch in range(num_epochs):
    model_v2.train()
    train_loss = 0.0
    for champion_ids, other_features, labels in train_loader:
        champion_ids, other_features, labels = champion_ids.to(device), other_features.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model_v2(champion_ids, other_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * champion_ids.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Validation
    model_v2.eval()
    val_loss = 0.0
    with torch.no_grad():
        for champion_ids, other_features, labels in val_loader:
            champion_ids, other_features, labels = champion_ids.to(device), other_features.to(device), labels.to(device)
            outputs = model_v2(champion_ids, other_features)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * champion_ids.size(0)
    
    val_loss /= len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Save the trained model
torch.save(model_v2.state_dict(), 'lol_itemization_model_v2.pth')

print("Training complete. Model saved as 'lol_itemization_model_v2.pth'")

In [7]:
class RevisedLoLItemizationModel(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super(RevisedLoLItemizationModel, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.layers(x)

In [8]:
def combine_features(X_champion, other_features_df):
    # champioons to series
    champion_series = pd.Series(X_champion, index=other_features_df.index, name='champion_id')
    
    # combine together
    combined_features = pd.concat([champion_series, other_features_df], axis=1)
    
    return combined_features

X_combined = combine_features(X_champion, other_features_df)

# PyTorch tensors
X_combined_tensor = torch.FloatTensor(X_combined.values)
y_tensor = torch.FloatTensor(items_df.values)

full_dataset = TensorDataset(X_combined_tensor, y_tensor)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

batch_size = 32  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

input_dim = X_combined.shape[1]
output_dim = items_df.shape[1]
num_champions = len(champion_encoder.classes_)
num_other_features = other_features_df.shape[1]

model = RevisedLoLItemizationModel(input_dim, output_dim)

device = torch.device("cpu")
model.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
    
    val_loss /= len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Save the model
torch.save(model.state_dict(), 'simplified_lol_itemization_model.pth')
print("Training complete. Model saved as 'simplified_lol_itemization_model.pth'")

Epoch 1/20, Train Loss: 0.0734, Val Loss: 0.0658
Epoch 2/20, Train Loss: 0.0685, Val Loss: 0.0650
Epoch 3/20, Train Loss: 0.0678, Val Loss: 0.0790
Epoch 4/20, Train Loss: 0.0674, Val Loss: 0.0697
Epoch 5/20, Train Loss: 0.0671, Val Loss: 0.0639
Epoch 6/20, Train Loss: 0.0669, Val Loss: 0.0645
Epoch 7/20, Train Loss: 0.0667, Val Loss: 0.0666
Epoch 8/20, Train Loss: 0.0666, Val Loss: 0.0647
Epoch 9/20, Train Loss: 0.0665, Val Loss: 0.0642
Epoch 10/20, Train Loss: 0.0664, Val Loss: 0.0654
Epoch 11/20, Train Loss: 0.0663, Val Loss: 0.0651
Epoch 12/20, Train Loss: 0.0662, Val Loss: 0.0641
Epoch 13/20, Train Loss: 0.0662, Val Loss: 0.0640
Epoch 14/20, Train Loss: 0.0661, Val Loss: 0.0642
Epoch 15/20, Train Loss: 0.0660, Val Loss: 0.0638
Epoch 16/20, Train Loss: 0.0660, Val Loss: 0.0636
Epoch 17/20, Train Loss: 0.0660, Val Loss: 0.0761
Epoch 18/20, Train Loss: 0.0659, Val Loss: 0.0628
Epoch 19/20, Train Loss: 0.0659, Val Loss: 0.0625
Epoch 20/20, Train Loss: 0.0659, Val Loss: 0.0631
Training 

In [17]:
def test_model_simplified(model, test_loader):
    model.eval()
    with torch.no_grad():
        inputs, labels = next(iter(test_loader))  # Only two values: inputs and labels
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Pass the inputs through the model
        outputs = model(inputs)
        predicted_items = outputs.cpu().numpy()

    # Print the predicted items
    print("Predicted items:")
    for i, item in enumerate(predicted_items[0]):
        if item >= 0.2:  # Assuming a threshold of 0.2 for the item prediction
            print(get_item_name(i, item_encoder, items))

            
for i in range(1, 6):  
    print(f"Example {i}:")
    
    # Decode inputs for display (optional)
    champion_name, other_features = decode_input(X_champion_val[i-1:i], X_other_val[i-1:i], champion_encoder, other_features_df)
    print("Champion:", champion_name)
    
    # Prepare test data by combining champion and other features
    champion_series = pd.Series(X_champion_val[i-1:i].cpu().numpy(), index=other_features_df.index[:1], name='champion_id')
    combined_features_df = pd.concat([champion_series, other_features_df.iloc[i-1:i]], axis=1)
    
    # Convert to tensor
    inputs_tensor = torch.tensor(combined_features_df.values, dtype=torch.float32).to(device)
    
    # Create a DataLoader for batch processing
    val_data = TensorDataset(inputs_tensor, y_val[i-1:i])
    test_loader = DataLoader(val_data, batch_size=1)
    
    # Test the model
    test_model_simplified(model, test_loader)

Example 1:
Champion: Jayce
Predicted items:
Doran's Shield
Plated Steelcaps
Sunfire Aegis
Bramble Vest
Mercury's Treads
Stealth Ward
Oracle Lens
Iceborn Gauntlet
Example 2:
Champion: Veigar


AssertionError: Size mismatch between tensors